In [ ]:
using ITensors
using ITensorTDVP
using Plots
using Observers
using LinearAlgebra

In [ ]:
N  = 21     # Number of spins
J  = 1.0    # ZZ interaction strength
hx = 1.05   # X-field 
hz = 0.5    # Z-field
δt = 0.05   # Time-step for evolution
T  = 10.0    # Total time
χ  = 32     # Max link dimension allowed
t = 0.05    # tunneling constant
U = 1.0     # on-site repulsion
μ = 0.5     # chemical potential

In [ ]:
sitesext = siteinds("Boson",2*N); # Make 2N Bosonic indices defining system + ancilla

A = ops(sitesext, [("a", n) for n in 1:(2*N)]);  # Annihilation operator
Adag = ops(sitesext, [("adag", n) for n in 1:(2*N)]);  # Creation operator
Adag_2 = ops(sitesext, [("adag * adag", n) for n in 1:(2*N)]) 
Adag_3 = ops(sitesext, [("adag * adag * adag", n) for n in 1:(2*N)])
;

In [ ]:
# ITensors doesn't include the identity operator as standard so construct it:
Id = Vector{ITensor}(undef,2*N)
for i =1:(2*N)
    iv = sitesext[i]
    ID = ITensor(iv', dag(iv));
    for j in 1:ITensors.dim(iv)
        ID[iv' => j, iv => j] = 1.0
    end
    Id[i] = ID
end;

In [ ]:
# Construct the identity vacuum state:

Ivac = MPS(sitesext, "0") # All up spins initial state
# 1:2:(2*N) from 1 to 2N in steps of 2
# we think Id[n].. does |11>, A[n].. does |00>, Adag[n].. does |22>, Adag_2[n]... does |33> and Adag_3[n].. does |44>
gates = [(Id[n]*Id[n+1] + A[n]*A[n+1] + Adag[n]*Adag[n+1] + Adag_2[n]*Adag_2[n+1] + Adag_3[n]*Adag_3[n+1])
                                                for n in 1:2:(2*N)]; 
# Maps |00> => |00> + |11> + |22> + |33> + |44>
Ivac = apply(gates, Ivac; cutoff=1e-15); # Note we have no 1/sqrt(2) normalisation

In [ ]:
H_op = OpSum()

#################
# Bose Hubbard Hamiltonian
####################
for i=1:2*(N-1)
    H_op += (-1)^(i) * t,"adag", i, "a", i+2 # system and system + 1
    H_op += (-1)^(i) * t,"a", i, "adag", i+2 # system and system + 1
    # needs to be -t Σ(b†i bj + bi b†j) so (-1)^(i) as opposed to ^(i-1)
end

for i=1:2*N
    H_op += (-1)^(i-1) * U/2, "n",i,"n", i
    H_op += (-1)^(i) * U/2, "n",i # -1 factor
    H_op += (-1)^(i) * μ, "n", i
end 

# Convert these terms to an MPO
HC = MPO(H_op,sitesext);

In [ ]:
# Define observable for scrambling:

A_op = OpSum()
A_op += 1.0,"n",2*floor(Int,N/2+1)-1  # Sx operator in the middle of the system
A = MPO(A_op,sitesext);                # Build the MPO from these terms
Avec = apply(A, Ivac; cutoff=1e-15);   # Compute |A> = A|I>

In [ ]:
# Define function for computing entanglement entropy

function entanglement_entropy(ψ)
    # Compute the von Neumann entanglement entropy across each bond of the MPS
        N = length(ψ)
        SvN = zeros(N)
        psi = ψ
        for b=1:N
            psi = orthogonalize(psi, b)
            if b==1
                U,S,V = svd(psi[b] , siteind(psi, b))
            else
                U,S,V = svd(psi[b], (linkind(psi, b-1), siteind(psi, b)))
            end
            p = diag(S).^2               # Extract square of Schmidt coefficients
            p = p ./ sum(p)              # Normalise to a probability dist
            SvN[b] = -sum(p .* log2.(p)) # Compute Shannon entropy
        end
        return SvN
    end;

In [ ]:
SvN_init = entanglement_entropy(Avec);

In [ ]:
# Define observer functions for TDVP:

function current_time(; current_time, bond, half_sweep)
    if bond == 1 && half_sweep == 2
      return real(-im*current_time)
    end
      
    return nothing
end
  
function measure_SvN(; psi, bond, half_sweep)
    if bond == 1 && half_sweep == 2
      return entanglement_entropy(psi)-SvN_init
    end
    return nothing
end;
  
function measure_linkdim(; psi, bond, half_sweep)
    if bond == 1 && half_sweep == 2
      return maxlinkdim(psi)
    end
    return nothing
end;

In [ ]:
# Perform TDVP evolution of |A(t)>:

obs = Observer("times" => current_time, "SvN" => measure_SvN, "chi" => measure_linkdim)

# d|A(t)>/dt = i HC |A(t)> so |A(t)> = exp(i t HC)|A(0)> 

ψf = tdvp(HC, im * T, Avec; 
          time_step = im * δt,
          normalize = false, 
          maxdim = χ,
          cutoff = 1e-10,
          outputlevel=1,
          (observer!)=obs)

# Extract results from time-step observations
times = obs.times
SvN = obs.SvN
chi = obs.chi;

In [ ]:
# Plot the entanglement entropy of each bond for system + ancilla:
gr() 
heatmap(1:(2*N), times, reduce(vcat,transpose.(SvN)), c = :seaborn_rocket_gradient)

In [ ]:
using Plots.PlotMeasures
# Plot the entanglement entropy for bonds separating system + ancilla pairs:
gr()
S = reduce(vcat,transpose.(SvN))[:,2:2:(2*N)]
heatmap(1:N, times, S, c = :sunset,left_margin=20px, right_margin=20px, top_margin=20px, framestyle=:box)

In [ ]:
# Plot entanglement entropy of bonds between system + ancilla pairs:
gr()
S = reduce(vcat,transpose.(SvN))[:,1:2:(2*N)]
heatmap(1:N, times, S, c = :thermal)

In [ ]:
# Plot the growth in the maximum link dimension with time:
plot(times, chi, label=false, linecolor="purple")  